In [13]:
# from scipy.sparse import csr_matrix

### Входные данные

#### Загрузка

In [14]:
# !unzip capstone_user_identification.zip

In [15]:
# from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import pickle

In [16]:
# PATH_TO_DATA = '../capstone_user_identification'
PATH_TO_DATA = '../input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2'

times = ['time%s' % i for i in range(1, 11)]
sites = ['site%s' % i for i in range(1, 11)]
df_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id', parse_dates=times)
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id', parse_dates=times)
df_train.shape, df_test.shape

((253561, 21), (82797, 20))

Словарь сайтов

In [17]:
with open(os.path.join(PATH_TO_DATA, 'site_dic.pkl'), 'rb') as f:
    site2id  = pickle.load(f)
id2site = {v:k for (k, v) in site2id.items()}
id2site[0] = 'unknown'

#### Обработка

In [18]:
df_train.sort_values(by='time1', inplace=True)

y_train = df_train["target"].astype('int').values

### Feature engineering

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
# from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

import numpy as np
import calendar

#### Sites

In [20]:
class SiteFeaturesPreparator(BaseEstimator, TransformerMixin):
    """
    Fill NaN with zero values;
    Prepare a (Count)Vectorizer friendly 2D-list from data.
    """
    def __init__(self):
        super().__init__()
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        sites = ['site%s' % i for i in range(1, 11)]
        # Convert dataframe rows to strings
#         return X[sites].fillna(0).astype('int')#.apply(lambda row: ' '.join([self.id2site[i] for i in row]), axis=1).tolist()
        return X[sites].fillna(0).astype('int').apply(lambda row: ' '.join([str(site_id) for site_id in row]), axis=1).tolist()

In [21]:
vectorizer_pipeline = Pipeline([
    ("preparator", SiteFeaturesPreparator()),
    ("vectorizer", CountVectorizer(ngram_range=(1, 2), max_features=20000))  # tokenizer=lambda s: s.split()
#     ("vectorizer", TfidfVectorizer(ngram_range=(1, 10), max_features=70000))
])

In [22]:
%%time
sites_train_vectorized = vectorizer_pipeline.fit_transform(df_train)
sites_train_vectorized

CPU times: user 11.5 s, sys: 65.9 ms, total: 11.6 s
Wall time: 11.6 s


<253561x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 2498250 stored elements in Compressed Sparse Row format>

In [97]:
df_train[sites]

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54843,56,55.0,56.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN
77292,946,946.0,951.0,946.0,946.0,945.0,948.0,784.0,949.0,946.0
114021,945,948.0,949.0,948.0,945.0,946.0,947.0,945.0,946.0,946.0
146670,947,950.0,948.0,947.0,950.0,952.0,946.0,951.0,946.0,947.0
...,...,...,...,...,...,...,...,...,...,...
12224,50,50.0,48.0,49.0,48.0,52.0,52.0,49.0,303.0,304.0
164438,4207,753.0,753.0,52.0,50.0,4207.0,3346.0,3359.0,3346.0,38.0
12221,52,3346.0,784.0,784.0,3346.0,979.0,3324.0,7330.0,3594.0,3329.0


In [101]:
class SitesFeaturesAdder(BaseEstimator, TransformerMixin):
    """
    Add new attributes to training and test set.
    """ 
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        sites = ['site%s' % i for i in range(1, 11)]
        
        n_sites = X[sites].apply(lambda session: session.count(), axis=1)
        n_unique = X[sites].fillna(0).apply(lambda session: session.nunique(), axis=1)
        
        site_counts = {}
        for session in X[sites].fillna(0).values:
            for site in session:
                if site in site_counts:
                    site_counts[site] += 1
                else:
                    site_counts[site] = 1
        site_counts.pop(0)
        top_n = [k for k, _ in sorted(site_counts.items(), key=lambda item: item[1], reverse=True)][:10]
        has_top10 = X[sites].fillna(0).apply(lambda session: session.isin(top_n).any().astype('int'), axis=1)
        
        features_df = pd.DataFrame({
            'n_sites': n_sites,
            'n_unique': n_unique,
            'has_top10': has_top10,
        })
        
        return features_df
    
all_site_features = ['n_sites', 'n_unique', 'has_top10']

In [102]:
%%time
site_features_pipeline = Pipeline([
    ("sites_adder", SitesFeaturesAdder()),
    ("scaler", StandardScaler())
])

sites_train = site_features_pipeline.fit_transform(df_train)
sites_train = pd.DataFrame(sites_train, columns=all_site_features)
sites_train

CPU times: user 1min 40s, sys: 499 ms, total: 1min 40s
Wall time: 1min 40s


,n_sites,n_unique,has_top10
0,-4.328969,-1.126296,-1.400389
1,-3.177031,-1.126296,-1.400389
2,0.278784,0.112680,-1.400389
3,0.278784,-0.300312,-1.400389
4,0.278784,0.112680,-1.400389
...,...,...,...
253556,0.278784,0.112680,0.714087
253557,0.278784,0.525672,0.714087
253558,0.278784,0.938664,0.714087
253559,0.278784,0.525672,-1.400389


#### Times

In [54]:
class TimesFeaturesAdder(BaseEstimator, TransformerMixin):
    """
    Add new attributes to training and test set.
    """ 
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        # intraday features
        hour = X['time1'].apply(lambda ts: ts.hour)
        minutes = X['time1'].apply(lambda ts: ts.minute)
        hour_minutes = hour + minutes / 60
        dummy_hours = pd.get_dummies(hour)
        
        morning = ((hour >= 7) & (hour <= 11)).astype('int')
        day = ((hour >= 12) & (hour <= 18)).astype('int')
        evening = ((hour >= 19) & (hour <= 23)).astype('int')
        
        sin_hour = np.sin(2 * np.pi * hour / 24)
        cos_hour = np.cos(2 * np.pi * hour / 24)
        
        # date
        month = X['time1'].apply(lambda ts: ts.month)
        year = X['time1'].apply(lambda ts: ts.year)
        year_month = year + month
        summer = ((month >= 6) & (month <= 8)).astype('int')
        
        weekday = X['time1'].apply(lambda ts: ts.weekday())
        dummy_weekdays = pd.get_dummies(data=weekday)
        weekend = np.isin(weekday, [5, 6])
        
        times = ['time%s' % i for i in range(1, 11)]
        session_duration = (X[times].max(axis=1) - X[times].min(axis=1)).astype('timedelta64[ms]').astype(int) #** 0.2
        
        features_df = pd.DataFrame({
            'hour': hour,
            'morning': morning,
            'day': day,
            'evening': evening,
            'sin_hour': sin_hour,
            'cos_hour': cos_hour,
            'minutes': minutes,
            'hour_minutes': hour_minutes,
            'month': month,
            'year': year,
            'year_month': year_month,
            'summer': summer,
            'weekday': weekday,
            'weekend': weekend,
            'session_duration': session_duration,
        })

        return pd.concat([features_df, dummy_hours, dummy_weekdays], axis=1)
    
all_time_features = ['hour', 'morning', 'day', 'evening', 'sin_hour', 'cos_hour',
                'minutes', 'hour_minutes', 'month', 'year', 'year_month',
                'summer', 'weekday', 'weekend', 'session_duration'] + list(range(7, 24)) + list(calendar.day_name)

In [55]:
%%time
time_features_pipeline = Pipeline([
    ("times_adder", TimesFeaturesAdder()),
    ("scaler", StandardScaler())
])

times_train = time_features_pipeline.fit_transform(df_train)
times_train = pd.DataFrame(times_train, columns=all_time_features)
times_train

CPU times: user 11.4 s, sys: 749 ms, total: 12.2 s
Wall time: 12.2 s


,hour,morning,day,evening,sin_hour,cos_hour,minutes,hour_minutes,month,year,...,21,22,23,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,-1.357366,1.039061,-0.98128,-0.171577,1.406483,0.644886,-1.400885,-1.487757,-0.943567,-1.547749,...,-0.082278,-0.076284,-0.072862,-0.436072,-0.487314,-0.532230,-0.459143,-0.440082,3.879327,-0.172561
1,-1.357366,1.039061,-0.98128,-0.171577,1.406483,0.644886,0.451667,-1.318638,-0.943567,-1.547749,...,-0.082278,-0.076284,-0.072862,-0.436072,-0.487314,-0.532230,-0.459143,-0.440082,3.879327,-0.172561
2,-1.357366,1.039061,-0.98128,-0.171577,1.406483,0.644886,1.204266,-1.249933,-0.943567,-1.547749,...,-0.082278,-0.076284,-0.072862,-0.436072,-0.487314,-0.532230,-0.459143,-0.440082,3.879327,-0.172561
3,-1.357366,1.039061,-0.98128,-0.171577,1.406483,0.644886,1.204266,-1.249933,-0.943567,-1.547749,...,-0.082278,-0.076284,-0.072862,-0.436072,-0.487314,-0.532230,-0.459143,-0.440082,3.879327,-0.172561
4,-1.357366,1.039061,-0.98128,-0.171577,1.406483,0.644886,1.204266,-1.249933,-0.943567,-1.547749,...,-0.082278,-0.076284,-0.072862,-0.436072,-0.487314,-0.532230,-0.459143,-0.440082,3.879327,-0.172561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253556,3.390349,-0.962408,-0.98128,5.828302,-0.399203,5.131687,0.220098,3.416696,-0.137395,0.646100,...,-0.082278,-0.076284,13.724647,-0.436072,-0.487314,1.878888,-0.459143,-0.440082,-0.257777,-0.172561
253557,3.390349,-0.962408,-0.98128,5.828302,-0.399203,5.131687,0.277990,3.421981,-0.137395,0.646100,...,-0.082278,-0.076284,13.724647,-0.436072,-0.487314,1.878888,-0.459143,-0.440082,-0.257777,-0.172561
253558,3.390349,-0.962408,-0.98128,5.828302,-0.399203,5.131687,0.509559,3.443121,-0.137395,0.646100,...,-0.082278,-0.076284,13.724647,-0.436072,-0.487314,1.878888,-0.459143,-0.440082,-0.257777,-0.172561
253559,3.390349,-0.962408,-0.98128,5.828302,-0.399203,5.131687,0.509559,3.443121,-0.137395,0.646100,...,-0.082278,-0.076284,13.724647,-0.436072,-0.487314,1.878888,-0.459143,-0.440082,-0.257777,-0.172561


In [46]:
class TimeDiffsFeaturesAdder(BaseEstimator, TransformerMixin):
    """
    Add new attributes to training and test set.
    """ 
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        def get_time_diff(row): 
            time_length = row.shape[0] - 1 
            time_diff = [0] * time_length 
            i = 0
            while (i < time_length) and pd.notnull(row[i+1]):
                time_diff[i] = (row[i+1] - row[i]) / np.timedelta64(1, 's') 
                i += 1 
            return time_diff
        
        time_diff = []
        times = ['time%s' % i for i in range(1, 11)]
        for row in df_train[times].values:
            time_diff.append(get_time_diff(row))    
        
        return time_diff

In [52]:
%%time
time_diffs_features_pipeline = Pipeline([
    ("times_adder", TimeDiffsFeaturesAdder()),
#     ("scaler", StandardScaler())
])

time_diffs_train = time_diffs_features_pipeline.fit_transform(df_train)
diff_names = [f'time_diff_{i}' for i in range(1, 10)]
time_diffs_train = pd.DataFrame(time_diffs_train, columns=diff_names, index=df_train.index)
time_diffs_train

CPU times: user 29.3 s, sys: 152 ms, total: 29.5 s
Wall time: 29.5 s


,time_diff_1,time_diff_2,time_diff_3,time_diff_4,time_diff_5,time_diff_6,time_diff_7,time_diff_8,time_diff_9
session_id,,,,,,,,,
21669,-0.191229,-0.180434,-0.186631,-0.193139,-0.192809,-0.193165,-0.192025,-0.190596,-0.190791
54843,-0.191229,16.706515,-0.165022,-0.193139,-0.192809,-0.193165,-0.192025,-0.190596,-0.190791
77292,-0.181131,-0.170968,-0.186631,-0.180395,-0.192809,-0.193165,-0.192025,-0.176180,-0.190791
114021,-0.191229,-0.170968,-0.186631,-0.193139,-0.192809,-0.179357,-0.192025,-0.190596,-0.175945
146670,-0.191229,-0.180434,-0.175826,-0.193139,-0.192809,-0.193165,-0.177680,-0.190596,-0.190791
...,...,...,...,...,...,...,...,...,...
12224,-0.181131,-0.152036,-0.186631,-0.180395,-0.192809,-0.179357,-0.192025,-0.147347,-0.146253
164438,-0.181131,-0.170968,-0.175826,-0.193139,0.572155,-0.013667,0.424788,0.241888,0.269431
12221,-0.171032,-0.152036,-0.132608,-0.142162,-0.166431,-0.041282,-0.192025,-0.190596,-0.175945


In [ ]:
#         # day of the week features
#         #     is_weekend = np.isin(weekday, [0, 2, 6])
        
#         session_duration = (X[times].max(axis=1) - X[times].min(axis=1)).astype('timedelta64[ms]').astype(int) #** 0.2

In [ ]:
# class ScaledAttributesAdder(BaseEstimator, TransformerMixin):
#     """
#     Add new features, that should be scaled.
#     """
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X, y=None):
#         # session time features
#         times = ['time%s' % i for i in range(1, 11)]
#         sites = ['site%s' % i for i in range(1, 11)]
        
#         # session duration: take to the power of 1/5 to normalize the distribution
#         session_duration = (X[times].max(axis=1) - X[times].min(axis=1)).astype('timedelta64[ms]').astype(int) #** 0.2
        
#         # number of sites visited in a session
#         number_of_sites = X[times].isnull().sum(axis=1).apply(lambda x: 10 - x)
        
#         # average time spent on one site during a session
#         time_per_site = (session_duration / number_of_sites) #** 0.2
        
# #         unique_sites = X[sites].apply(lambda sites: sites.nunique(), axis=1)
        
#         X = np.c_[session_duration.values]
#         return X

In [ ]:
# scaled_attributes_pipeline = Pipeline([
#     ("adder", ScaledAttributesAdder()),
#     ("scaler", StandardScaler())
# ])

# feature_pipeline = FeatureUnion(transformer_list=[
#     ('attributes_pipeline', attributes_pipeline),
#     ('scaled_attributes_pipeline', scaled_attributes_pipeline)
# ])

### Feature selection

In [57]:
from scipy.sparse import hstack

##############################################################

In [58]:
print(all_time_features)

['hour', 'morning', 'day', 'evening', 'sin_hour', 'cos_hour', 'minutes', 'hour_minutes', 'month', 'year', 'year_month', 'summer', 'weekday', 'weekend', 'session_duration', 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [103]:
print(all_site_features)

['n_sites', 'n_unique', 'has_top10']


In [16]:
list(calendar.day_name)

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [116]:
selected_time_features = ['morning', 'day', 'evening', 14, 15, 'year', 'Monday', 'Sunday', 'session_duration']
selected_site_features = ['has_top10']

X_train = hstack([sites_train_vectorized, times_train[selected_time_features], sites_train[selected_site_features]]) # time_diffs_train[['time_diff_1', 'time_diff_2']]
X_train

<253561x20010 sparse matrix of type '<class 'numpy.float64'>'
	with 5033860 stored elements in COOrdinate format>

##############################################################

### Модель

In [60]:
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

#### Обучение и подбор гиперпараметров

In [61]:
model = SGDClassifier(loss='log', random_state=2179, n_jobs=-1)
time_split = TimeSeriesSplit(n_splits=10)

In [ ]:
# grid = {
#     'penalty': ['l2'],
#     'alpha': np.linspace(3e-05, 5e-05, 11)
# }

In [ ]:
# %%time

# search = GridSearchCV(model, param_grid=grid, cv=time_split, scoring='roc_auc')
# search.fit(X_train, y_train)

In [ ]:
# search.best_params_, search.best_score_

##############################################################

In [117]:
%%time

cv_scores = cross_val_score(model, X_train, y_train, cv=time_split, scoring='roc_auc', n_jobs=-1)
print(cv_scores.mean(), cv_scores.std())
cv_scores

0.9384757285805904 0.03973406705806397
CPU times: user 253 ms, sys: 49.1 ms, total: 302 ms
Wall time: 2.44 s


array([0.91865168, 0.8430592 , 0.96593484, 0.95720985, 0.95430544,
       0.96995075, 0.8905313 , 0.95946189, 0.95970742, 0.96594491])

##############################################################

Опции:

- CountVectorizer ngram_range max_features
    - (1, 1)
        - max-k: 0.8530563468012738, 0.08925871896536618
        - 20000: 0.8531100483866421, 0.08919480787349321
        - 10000: 0.8539501616208991, 0.08875887000416063
        - 5000: 0.8529069996789713, 0.08888832394047236
    - (1, 2)
        - max-k: 0.8577666221636842, 0.09371826647019514
        - 50000: 0.858414628277273, 0.09308624515067462
        - 20000: 0.8600043484189314, 0.0901820258132247
        - 10000: 0.8565645409290898, 0.09086786443064755
    - (1, 3)
        - max-k: 0.8560039171386127, 0.09631952839070929
        - 50000: 0.859467798898789, 0.09245656399159023
        - 20000: 0.8598701871410264 0.09139402713364686
        - 10000: 0.8560423530991228 0.09031323488865993
    - (1, 4)
        - max-k: 0.8533973079027204 0.10052454183974614
        - 20000: 0.8455330939583249 0.08401555400318593
    - (1, 9)
        - 70000: .8604293420872524 0.09704219048563271
    - (1, 10)
        - 20000: 0.8419623744960409 0.0864989715893941
        - 50000: 0.859496836616332 0.09749267442904218
        - 70000: 0.8604687084516304 0.0970295359566777
        - 100000: 0.8596446298744558 0.09781978585361331
- TfidfVectorizer ngram_range max_features
    - (1, 1)
        - max-k: 0.8391530980059445, 0.08120401396593267
        - 20000: 0.8399308212634944 0.08111430205756767
    - (1, 2)
        - max-k: 0.8294502665752119, 0.08421253009084147
        - 20000: 0.8398228277779543 0.07824510376730294
    - (1, 3)
        - max-k: 0.8230380243367905, 0.08660127058036823
    - (1, 10):
        - 70000: 0.8359176499242273 0.08504483506085675
        
Выбираем CountVectorizer(ngram_range=(1, 2), max_features=20000)

0.9315069358401142,
 0.044345068769381366 0 2 6

### Тест

In [ ]:
# import eli5
# import calendar

def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                               
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
%%time

sites_test_vectorized = vectorizer_pipeline.transform(df_test)
times_test = time_features_pipeline.transform(df_test)
times_test = pd.DataFrame(times_test, columns=all_features)
X_test = hstack([sites_test_vectorized, times_test[selected_features]])
X_test

In [ ]:
# search.best_estimator_.fit(X_train, y_train)
# logit_test_pred_proba = search.best_estimator_.predict_proba(X_test)
model.fit(X_train, y_train)
logit_test_pred_proba = model.predict_proba(X_test)

In [ ]:
# pd.DataFrame({'feature': feature_names, 'coef': search.best_estimator_.coef_.flatten()[-len(feature_names):]})

In [ ]:
# eli5.show_weights(estimator=search.best_estimator_, feature_names=vectorizer_pipeline['vectorizer'].get_feature_names() \
#                   + feature_names, top=30)

In [ ]:
write_to_submission_file(logit_test_pred_proba[:, 1], 'result.csv')